<a href="https://colab.research.google.com/github/MichelleFn/datamining/blob/main/dataming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn import datasets
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder='/content/drive/My Drive/Data_Mining'
!ls '/content/drive/My Drive/Data_Mining'

In [ ]:
# Φόρτωση δεδομένων Ιουνίου df και Ιουλίου df1
df = pd.read_csv(folder+'/listings_june.csv')
df1 = pd.read_csv(folder+'/listings_july.csv')

ΑΡΧΙΚΗ ΕΠΕΞΕΡΓΑΣΙΑ


In [ ]:
to_drop = ['host_name','host_id','name','id','neighbourhood_group','last_review']

In [ ]:
df.drop(to_drop, inplace=True, axis=1)

In [ ]:
df1.drop(to_drop, inplace=True, axis=1)

In [ ]:
mean_review = int(df['reviews_per_month'].mean())
mean_review
df['reviews_per_month'] = df['reviews_per_month'].fillna(mean_review)

In [ ]:
mean_review = int(df1['reviews_per_month'].mean())
mean_review
df1['reviews_per_month'] = df1['reviews_per_month'].fillna(mean_review)

In [ ]:
df['cost_per_stay'] = df['price'] * df['minimum_nights']

In [ ]:
df1['cost_per_stay'] = df1['price'] * df1['minimum_nights']

In [ ]:
df = df[df['minimum_nights'] <= 10]

In [ ]:
df1 = df1[df1['minimum_nights'] <= 10]

In [ ]:
df['target_price_group'] = 0
df['target_price_group'] = np.where((df['price'] > 0) & (df['price'] <= 50), 50, df['target_price_group'])
df['target_price_group'] = np.where((df['price'] > 50) & (df['price'] <= 101), 100, df['target_price_group'])
df['target_price_group'] = np.where((df['price'] > 101) & (df['price'] <= 201), 200, df['target_price_group'])
df['target_price_group'] = np.where((df['price'] > 200) & (df['price'] <= 500), 500, df['target_price_group'])

In [ ]:
df1['target_price_group'] = 0
df1['target_price_group'] = np.where((df1['price'] > 0) & (df1['price'] <= 50), 50, df1['target_price_group'])
df1['target_price_group'] = np.where((df1['price'] > 50) & (df1['price'] <= 101), 100, df1['target_price_group'])
df1['target_price_group'] = np.where((df1['price'] > 101) & (df1['price'] <= 201), 200, df1['target_price_group'])
df1['target_price_group'] = np.where((df1['price'] > 200) & (df1['price'] <= 500), 500, df1['target_price_group'])

In [ ]:
!pip install category_encoders

In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
df_encoded = encoder.fit_transform(df)
df_encoded

In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
df1_encoded = encoder.fit_transform(df1)
df1_encoded

In [ ]:
df_encoded[df_encoded['price'] > 500].groupby('price')['price'].count()
df_outliers = df_encoded[df_encoded['price'] < 500]

In [ ]:
df1_encoded[df1_encoded['price'] > 500].groupby('price')['price'].count()
df1_outliers = df_encoded[df_encoded['price'] < 500]

ΜΕΘΟΔΟΙ REGRESSION

In [ ]:
#linear regression model for july data
def LinearRegression(x_train,y_train,x_test,y_test):
 from sklearn.linear_model import LinearRegression

 linear_reg_model = LinearRegression().fit(x_train, y_train)
 y_pred = linear_reg_model.predict(x_test)  

 print("intercept: " ,linear_reg_model.intercept_)
 print("coefficceiance :", linear_reg_model.coef_)

 print("r2_score: ",r2_score(y_test, y_pred))
 print("mean_squared_error: " ,mean_squared_error(y_test,y_pred))
# print(y_test)
# print(y_pred)

In [ ]:
#decision tree regression for july data
def DecisionTreeRegression(x_train,y_train,x_test,y_test):
 from sklearn.tree import DecisionTreeRegressor

 d_tree_reg_model = DecisionTreeRegressor()
 d_tree_reg_model.fit(x_train,y_train)
 y_pred_tree = d_tree_reg_model.predict(x_test)
 print(y_test)
 print(y_pred_tree)

 print("mean squared error: ",mean_squared_error(y_test,y_pred_tree))
 print("r2_score:" ,r2_score(y_test,y_pred_tree))

In [ ]:
#random forest regression for july data
def RandomForestRegression(x_train,y_train,x_test,y_test):
  from sklearn.ensemble import RandomForestRegressor

  random_forest_reg_model = RandomForestRegressor(n_estimators=200, oob_score=True, random_state=0)
  random_forest_reg_model.fit(x_train, y_train)
  y_pred_rf = random_forest_reg_model.predict(x_test)
  print(y_pred_rf)
  print(y_test)
  print("mean squared error: ",mean_squared_error(y_test,y_pred_rf))
  print("r2_score:" ,r2_score(y_test,y_pred_rf))

In [ ]:
#regression with svr for july data
def SVRegression(x_train,y_train,x_test,y_test):
  from sklearn.svm import SVR
  svr_model = SVR()
  svr_model.fit(x_train,y_train)
  y_pred_svr = svr_model.predict(x_test)
  print(y_pred_svr)
  print(y_test)

  print("mean squared error: ",mean_squared_error(y_test,y_pred_svr))
  print("r2_score:" ,r2_score(y_test,y_pred_svr))

In [ ]:
from sklearn.model_selection import train_test_split
x_alt_out = df_outliers.loc[:, df_outliers.columns != 'price']
y_alt_out  = df_outliers[["price"]]
x_train_out, x_test_out, y_train_out, y_test_out = train_test_split(x_alt_out, y_alt_out, train_size=0.9, random_state=None)

In [ ]:
LinearRegression(x_train_out,y_train_out,x_test_out,y_test_out)

In [ ]:
DecisionTreeRegression(x_train_out,y_train_out,x_test_out,y_test_out)

In [ ]:
RandomForestRegression(x_train_out,y_train_out,x_test_out,y_test_out)

In [ ]:
SVRegression(x_train_out,y_train_out,x_test_out,y_test_out)

In [ ]:
from sklearn.model_selection import train_test_split
x_alt_out_july = df1_outliers.loc[:, df1_outliers.columns != 'price']
y_alt_out_july  = df1_outliers[["price"]]
x_train_out_july, x_test_out_july, y_train_out_july, y_test_out_july = train_test_split(x_alt_out_july, y_alt_out_july, train_size=0.9, random_state=None)

In [ ]:
LinearRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

In [ ]:
DecisionTreeRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

In [ ]:
RandomForestRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

In [ ]:
SVRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

ΜΕΘΟΔΟΙ ΚΑΤΗΓΟΡΙΟΠΟΙΗΣΗΣ

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#decision trees classifier
def DecisionTreeClassifier(x_train,y_train,x_test,y_test):
 from sklearn.tree import DecisionTreeClassifier
 dec_tree_clas = DecisionTreeClassifier(criterion='entropy',random_state=0) 
 dec_tree_clas.fit(x_train,y_train)
 y_pred_dec_tree_clas = dec_tree_clas.predict(x_test)
 percision = precision_score(y_test, y_pred_dec_tree_clas,average=None)
 recall = recall_score(y_test, y_pred_dec_tree_clas,average=None)
 f1 = f1_score(y_test, y_pred_dec_tree_clas,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)

#  print(confusion_matrix(y_test_class, y_pred_dec_tree_clas))
#  print(classification_report(y_test_class, y_pred_dec_tree_clas))
 print(accuracy_score(y_test, y_pred_dec_tree_clas))

In [ ]:
#random forest classification
def RandomForestClassification(x_train,y_train,x_test,y_test):
 from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
 random_forest_class_model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
 random_forest_class_model.fit(x_train, y_train)
 y_pred_rf_clas = random_forest_class_model.predict(x_test)
 percision = precision_score(y_test,y_pred_rf_clas,average=None)
 recall = recall_score(y_test, y_pred_rf_clas,average=None)
 f1 = f1_score(y_test,y_pred_rf_clas,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)
#  print(confusion_matrix(y_test_class, y_pred_rf_clas))
#  print(classification_report(y_test_class, y_pred_rf_clas))
 print(accuracy_score(y_test, y_pred_rf_clas))


In [ ]:
def SVClassification(x_train,y_train,x_test,y_test):
 from sklearn.svm import SVC
 SupportVectorClassModel = SVC()
 SupportVectorClassModel.fit(x_train,y_train)
 y_pred_sv_clas = SupportVectorClassModel.predict(x_test)

 percision = precision_score(y_test, y_pred_sv_clas,average=None)
 recall = recall_score(y_test,y_pred_sv_clas,average=None)
 f1 = f1_score(y_test, y_pred_sv_clas,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)

#  print(confusion_matrix(y_test_class, y_pred_sv_clas))
#  print(classification_report(y_test_class, y_pred_sv_clas))
 print(accuracy_score(y_test, y_pred_sv_clas))


In [ ]:
#knn classification
def knnClassification(x_train,y_train,x_test,y_test):
 from sklearn.preprocessing import StandardScaler
 scaler = StandardScaler()
 scaler.fit(x_train)

 x_train_scaled = scaler.transform(x_train)
 x_test_scaled = scaler.transform(x_test)

 from sklearn.neighbors import KNeighborsClassifier
 classifier = KNeighborsClassifier(n_neighbors=5)
 classifier.fit(x_train_scaled, y_train)
 y_pred_knn = classifier.predict(x_test_scaled)

 from sklearn.metrics import classification_report, confusion_matrix

 percision = precision_score(y_test, y_pred_knn,average=None)
 recall = recall_score(y_test, y_pred_knn,average=None)
 f1 = f1_score(y_test, y_pred_knn,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)
#  print(confusion_matrix(y_test_class, y_pred_knn))
#  print(classification_report(y_test_class, y_pred_knn))
 print(accuracy_score(y_test, y_pred_knn))


In [ ]:
from sklearn.model_selection import train_test_split
x_alt_class = df_outliers.loc[:, df_outliers.columns != 'target_price_group']
y_alt_class  = df_outliers[["target_price_group"]]
x_train_alt_class, x_test_alt_class, y_train_alt_class, y_test_alt_class = train_test_split(x_alt_class, y_alt_class, train_size=0.9, random_state=None)

In [ ]:
RandomForestClassification(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

In [ ]:
knnClassification(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

In [ ]:
DecisionTreeClassifier(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

In [ ]:
SVClassification(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

In [ ]:
from sklearn.model_selection import train_test_split
x_alt_class_july = df1_outliers.loc[:, df1_outliers.columns != 'target_price_group']
y_alt_class_july  = df1_outliers[["target_price_group"]]
x_train_alt_class_july, x_test_alt_class_july, y_train_alt_class_july, y_test_alt_class_july = train_test_split(x_alt_class_july, y_alt_class_july, train_size=0.9, random_state=None)

In [ ]:
knnClassification(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

In [ ]:
DecisionTreeClassifier(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

In [ ]:
SVClassification(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

In [ ]:
RandomForestClassification(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)